# Tensorflow - TensorRT INT8 Inference example from saved model

In this notebook, we demonstrate the process to create a TF-TensorRT optimized model from a Tensorflow *saved model*.
This notebook has been successfully tested in the NVIDIA NGC Tensorflow container `nvcr.io/nvidia/tensorflow:19.04-py3` that can be downloaded from http://ngc.nvidia.com.

### Data
We use the ImageNet dataset that has been stored in TFrecords format. Google provide an excellent all-in-one script for downloading and preparing the ImageNet dataset at https://github.com/tensorflow/models/blob/master/research/inception/inception/data/download_and_preprocess_imagenet.sh.

### Saved model
We will run this demonstration with a saved model from the Tensorflow Resnet model zoo https://github.com/tensorflow/models/tree/master/official/resnet.

To run this notebook, start the NGC TF container providing correct path to ImageNet validation data and a TF saved model:

```bash
nvidia-docker run -it -p 8888:8888 -v /path/to/image_net/:/data  -v /path/to/saved_model:/saved_model --name TFTRT nvcr.io/nvidia/tensorflow:19.04-py3
```

This repository can then be cloned to `/workspace`:

```bash
git clone https://github.com/vinhngx/tftrt-examples
```

Then start jupyter notebook within the container with:

```bash
cd tftrt-examples
jupyter notebook --ip 0.0.0.0 --port 8888  --allow-root
```

Connect to Jupyter notebook web interface from your local host http://localhost:8888. 

We first install some extra packages and external dependencies. 

In [ ]:
%%bash
pushd /workspace/nvidia-examples/tensorrt/tftrt/examples/object_detection
bash install_dependencies.sh;
popd

In [10]:
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
import numpy as np
import matplotlib.pyplot as plt
import os
import time

import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

## Data
We verify that the correct data folder has been mounted.

In [11]:
VALIDATION_DATA_DIR = "/data"

def get_files(data_dir, filename_pattern):
    if data_dir == None:
        return []
    files = tf.gfile.Glob(os.path.join(data_dir, filename_pattern))
    if files == []:
        raise ValueError('Can not find any files in {} with '
                         'pattern "{}"'.format(data_dir, filename_pattern))
    return files

calibration_files = get_files(VALIDATION_DATA_DIR, 'validation*')
print('There are %d calibration files. \n%s\n%s\n...'%(len(calibration_files), calibration_files[0], calibration_files[-1]))

There are 128 calibration files. 
/data/validation-00114-of-00128
/data/validation-00094-of-00128
...


## Saved model
If not already downloaded, we will be downloading and working with a ResNet-50 v2 (fp32, Accuracy 76.47%) from https://github.com/tensorflow/models/tree/master/official/resnet. 

In [12]:
%%bash
FILE=/saved_model/resnet_v2_fp32_savedmodel_NCHW.tar.gz
if [ -f $FILE ]; then
   echo "The file '$FILE' exists."
else
   echo "The file '$FILE' in not found. Downloading..."
   !wget -P /saved_model/ http://download.tensorflow.org/models/official/20181001_resnet/savedmodels/resnet_v2_fp32_savedmodel_NCHW.tar.gz 
fi


The file '/saved_model/resnet_v2_fp32_savedmodel_NCHW.tar.gz' exists.


In [11]:
!tar -xzvf /saved_model/resnet_v2_fp32_savedmodel_NCHW.tar.gz -C /saved_model 

./resnet_v2_fp32_savedmodel_NCHW/
./resnet_v2_fp32_savedmodel_NCHW/1538687196/
./resnet_v2_fp32_savedmodel_NCHW/1538687196/variables/
./resnet_v2_fp32_savedmodel_NCHW/1538687196/variables/variables.index
./resnet_v2_fp32_savedmodel_NCHW/1538687196/variables/variables.data-00000-of-00001
./resnet_v2_fp32_savedmodel_NCHW/1538687196/saved_model.pb


We employ TF `saved_model_cli` tool to inspect the model.

In [31]:
!saved_model_cli show --all --dir /saved_model/resnet_v2_fp32_savedmodel_NCHW/1538687196/


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input'] tensor_info:
        dtype: DT_FLOAT
        shape: (64, 224, 224, 3)
        name: input_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_INT64
        shape: (64)
        name: ArgMax:0
    outputs['probabilities'] tensor_info:
        dtype: DT_FLOAT
        shape: (64, 1001)
        name: softmax_tensor:0
  Method name is: tensorflow/serving/predict

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input'] tensor_info:
        dtype: DT_FLOAT
        shape: (64, 224, 224, 3)
        name: input_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_INT64
        shape: (6

## Helper functions
We define a few helper functions to read and preprocess Imagenet data from TFRecord files. 

In [13]:
def deserialize_image_record(record):
    feature_map = {
        'image/encoded':          tf.FixedLenFeature([ ], tf.string, ''),
        'image/class/label':      tf.FixedLenFeature([1], tf.int64,  -1),
        'image/class/text':       tf.FixedLenFeature([ ], tf.string, ''),
        'image/object/bbox/xmin': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymin': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/xmax': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymax': tf.VarLenFeature(dtype=tf.float32)
    }
    with tf.name_scope('deserialize_image_record'):
        obj = tf.parse_single_example(record, feature_map)
        imgdata = obj['image/encoded']
        label   = tf.cast(obj['image/class/label'], tf.int32)
        bbox    = tf.stack([obj['image/object/bbox/%s'%x].values
                            for x in ['ymin', 'xmin', 'ymax', 'xmax']])
        bbox = tf.transpose(tf.expand_dims(bbox, 0), [0,2,1])
        text    = obj['image/class/text']
        return imgdata, label, bbox, text

In [14]:
from preprocessing import inception_preprocessing, vgg_preprocessing
def preprocess(record):
        # Parse TFRecord
        imgdata, label, bbox, text = deserialize_image_record(record)
        #label -= 1 # Change to 0-based (don't use background class)
        try:    image = tf.image.decode_jpeg(imgdata, channels=3, fancy_upscaling=False, dct_method='INTEGER_FAST')
        except: image = tf.image.decode_png(imgdata, channels=3)

        image = vgg_preprocessing.preprocess_image(image, 224, 224, is_training=False)
        return image, label

In [15]:
def benchmark_saved_model(SAVED_MODEL_DIR, BATCH_SIZE):
    with tf.Session(graph=tf.Graph()) as sess:
        # Initialize all tfrecord paths
        dataset = tf.data.TFRecordDataset(calibration_files)    
        dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=preprocess, batch_size=BATCH_SIZE, num_parallel_calls=8))
        iterator = dataset.make_one_shot_iterator()
        next_element = iterator.get_next()

        tf.saved_model.loader.load(
            sess, [tf.saved_model.tag_constants.SERVING], SAVED_MODEL_DIR)

        num_hits = 0
        num_predict = 0
        print('Warming up for 10 batches...')
        for _ in range (10):
            image_data = sess.run(next_element)    
            img = image_data[0]
            label = image_data[1].squeeze()
            output = sess.run(['softmax_tensor:0'], feed_dict={"input_tensor:0": img})
            prediction = np.argmax(output[0], axis=1)
            num_hits += np.sum(prediction == label)
            num_predict += len(prediction)
            
        start_time = time.time()
        try:
            while True:        
                image_data = sess.run(next_element)    
                img = image_data[0]
                label = image_data[1].squeeze()
                output = sess.run(['softmax_tensor:0'], feed_dict={"input_tensor:0": img})
                prediction = np.argmax(output[0], axis=1)
                num_hits += np.sum(prediction == label)
                num_predict += len(prediction)
        except tf.errors.OutOfRangeError as e:
            pass

        print('Accuracy: %.2f%%'%(100*num_hits/num_predict)) 
        print('Inference speed: %.2f samples/s'%(num_predict/(time.time()-start_time)))

## Benchmarking naitive Tensorflow model

In [20]:
#Define some global variables
BATCH_SIZE = 64
SAVED_MODEL_DIR = "/saved_model/resnet_v2_fp32_savedmodel_NCHW/1538687196/FP32"

In [17]:
benchmark_saved_model(SAVED_MODEL_DIR, BATCH_SIZE)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
Warming up for 10 batches...
Accuracy: 76.36%
Inference speed: 1356.20 samples/s


## Benchmarking TF-TRT FP32  inference engine

In [24]:
FP32_SAVED_MODEL_DIR = SAVED_MODEL_DIR + "FP32"
!rm -rf $FP32_SAVED_MODEL_DIR

trt.create_inference_graph(
    input_graph_def=None,
    outputs=None,
    max_batch_size=BATCH_SIZE,
    input_saved_model_dir=SAVED_MODEL_DIR,
    output_saved_model_dir=FP32_SAVED_MODEL_DIR,
    precision_mode="FP32")

INFO:tensorflow:Running against TensorRT version 5.1.2
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /saved_model/resnet_v2_fp32_savedmodel_NCHW/1538687196/FP32FP32/saved_model.pb


In [26]:
benchmark_saved_model(FP32_SAVED_MODEL_DIR, BATCH_SIZE)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
Warming up for 10 batches...
Accuracy: 76.36%
Inference speed: 1365.15 samples/s


## Benchmarking TF-TRT FP16 inference engine

In [28]:
FP16_SAVED_MODEL_DIR = SAVED_MODEL_DIR + "FP16"
!rm -rf $FP16_SAVED_MODEL_DIR

trt.create_inference_graph(
    input_graph_def=None,
    outputs=None,
    max_batch_size=BATCH_SIZE,
    input_saved_model_dir=SAVED_MODEL_DIR,
    output_saved_model_dir=FP16_SAVED_MODEL_DIR,
    precision_mode="FP16")

INFO:tensorflow:Running against TensorRT version 5.1.2
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /saved_model/resnet_v2_fp32_savedmodel_NCHW/1538687196/FP32FP16/saved_model.pb


In [31]:
benchmark_saved_model(FP16_SAVED_MODEL_DIR, BATCH_SIZE)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
Warming up for 10 batches...
Accuracy: 76.36%
Inference speed: 1361.09 samples/s


## Creating TFTRT INT8 inference model

Creating TF-TRT INT8 inference model requires two steps:

- Step 1: creating the calibration graph, and run some training data through that graph for INT-8 calibration.

- Step 2: converting the calibration graph to the TF-TRT INT8 inference engine

### Step 1

In [37]:
tf.reset_default_graph()
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(
        sess, [tf.saved_model.tag_constants.SERVING], SAVED_MODEL_DIR)
    fp32_graph_def = sess.graph_def
len(fp32_graph_def.SerializeToString())    

#Now we create the TFTRT INT8 calibration graph
trt_int8_calib_graph = trt.create_inference_graph(
        input_graph_def=fp32_graph_def,
        outputs=['softmax_tensor:0'],
        max_batch_size=BATCH_SIZE,
        max_workspace_size_bytes=1<<32,
        precision_mode='INT8')

#Then calibrate it with 2 batchs of examples
N_runs=100
with tf.Session(graph=tf.Graph()) as sess:
    
    output_node = tf.import_graph_def(
        trt_int8_calib_graph,
        return_elements=['softmax_tensor'],
        name='trt')
    
    # Initialize all tfrecord paths
    dataset = tf.data.TFRecordDataset(calibration_files)
    dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=preprocess, batch_size=BATCH_SIZE, num_parallel_calls=8))
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()


    #print([n.name for n in tf.get_default_graph().as_graph_def().node])
    
    print('Calibrate model on calibration data...')
    num_hits = 0
    num_predict = 0
    for _ in range(N_runs):
            image_data = sess.run(next_element)    
            img = image_data[0]
            label = image_data[1].squeeze()
            output = sess.run(output_node[0].outputs[0], feed_dict={"trt/input_tensor:0": img})
            prediction = np.argmax(output, axis=1)
            num_hits += np.sum(prediction == label)
            num_predict += len(prediction)
    print('Calibration accuracy: %.2f%%'%(100*num_hits/num_predict)) 



INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
INFO:tensorflow:Running against TensorRT version 5.1.2
Calibrate model on calibration data...
Calibration accuracy: 75.38%


### Step 2

Now we convert the INT8 calibration graph to the final TF-TRT INT8 inference engine, and benchmark its performance. We will also be saving this engine to a *saved model*, ready to be served elsewhere.

In [34]:
INT8_SAVED_MODEL_DIR = SAVED_MODEL_DIR + "INT8"
!rm -rf $INT8_SAVED_MODEL_DIR

In [38]:
#Create Int8 inference model from the calibration graph and write to a saved session
trt_int8_calibrated_graph=trt.calib_graph_to_infer_graph(trt_int8_calib_graph)
output_node = tf.import_graph_def(
        trt_int8_calibrated_graph,
        return_elements=['softmax_tensor:0'],
        name='')

with tf.Session(graph=tf.Graph()) as sess:
    # Initialize all tfrecord paths
    dataset = tf.data.TFRecordDataset(calibration_files)    
    dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=preprocess, batch_size=BATCH_SIZE, num_parallel_calls=8))
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    output_node = tf.import_graph_def(
        trt_int8_calibrated_graph,
        return_elements=['softmax_tensor:0'],
        name='')

    print('Warming up for 10 batches...')
    for _ in range (10):
        image_data = sess.run(next_element)    
        img = image_data[0]
        output = sess.run(['softmax_tensor:0'], feed_dict={"input_tensor:0": img})
        
    print('Benchmarking TF-TRT INT8 inference engine...')
    num_hits = 0
    num_predict = 0
    start_time = time.time()
    try:
        while True:        
            image_data = sess.run(next_element)    
            img = image_data[0]
            label = image_data[1].squeeze()
            output = sess.run(['softmax_tensor:0'], feed_dict={"input_tensor:0": img})
            prediction = np.argmax(output[0], axis=1)
            num_hits += np.sum(prediction == label)
            num_predict += len(prediction)
    except tf.errors.OutOfRangeError as e:
        pass
            
    print('INT8 TF-TRT Accuracy: %.2f%%'%(100*num_hits/num_predict))
    print('INT8 TF-TRT Inference speed: %.2f samples/s'%(num_predict/(time.time()-start_time)))

    #Save model for serving
    tf.saved_model.simple_save(
        session=sess,
        export_dir=INT8_SAVED_MODEL_DIR,
        inputs={"input":tf.get_default_graph().get_tensor_by_name("import/input:0")},
        outputs={"softmax":output_node[0].outputs[0]},
        legacy_init_op=None
     )

      

FailedPreconditionError: Need to run graph with calibration data first!

## Benchmarking INT8 saved model

Finally we reload and verify the performance of the INT8 saved model.

In [ ]:
#benchmark int8 saved model
with tf.Session(graph=tf.Graph()) as sess:
    # Initialize all tfrecord paths
    dataset = tf.data.TFRecordDataset(calibration_files)    
    dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=preprocess, batch_size=BATCH_SIZE, num_parallel_calls=8))
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    tf.saved_model.loader.load(
        sess, [tf.saved_model.tag_constants.SERVING], SAVED_INT8_MODEL_DIR)
    
    print('Warming up for 10 batches...')
    for _ in range (10):
        image_data = sess.run(next_element)    
        img = image_data[0]
        output = sess.run(['import/resnet_v1_50/predictions/Softmax:0'], feed_dict={"import/input:0": img})
    
    print('Benchmarking TF-TRT INT8 inference engine...')
    num_hits = 0
    num_predict = 0
    start_time = time.time()
    try:
        while True:        
            image_data = sess.run(next_element)    
            img = image_data[0]
            label = image_data[1]
            output = sess.run(['import/resnet_v1_50/predictions/Softmax:0'], feed_dict={"import/input:0": img})
            prediction = np.argmax(output[0], axis=1)
            num_hits += np.sum(prediction == label)
            num_predict += len(prediction)
    except tf.errors.OutOfRangeError as e:
        pass
            
    print('INT8 TF-TRT Accuracy: %.2f%%'%(100*num_hits/num_predict))
    print('INT8 TF-TRT Inference speed: %.2f samples/s'%(num_predict/(time.time()-start_time)))


## Benchmarking with synthetic data

While benchmarking with real datasets, there are data reading and pre-processing procedures involved. As a result, the GPU is not fully loaded all the time. In this section, we test with synthetic data to test the throughput limit of the GPU.

In [ ]:
NUM_ITER = 100
dummy_input = np.random.random_sample((BATCH_SIZE,224,224,3))

fp32_graph = tf.Graph()
with fp32_graph.as_default():
    inc=tf.constant(dummy_input, dtype=tf.float32)
    dataset=tf.data.Dataset.from_tensors(inc)
    dataset=dataset.repeat()
    iterator=dataset.make_one_shot_iterator()
    next_element=iterator.get_next()
    out = tf.import_graph_def(
      graph_def=fp32_graph_def,
      input_map={"import/input":next_element},
      return_elements=[ "import/resnet_v1_50/predictions/Softmax"]
    )

with tf.Session(graph=fp32_graph) as sess:
    print('Warming up for 10 batches...')
    for _ in range(10):
        sess.run(out)
    
    print('Benchmarking...')
    start_time = time.time()
    for _ in range(NUM_ITER):
        sess.run(out)
    print('Naitive FP32 Inference speed: %.2f samples/s'%(NUM_ITER*BATCH_SIZE/(time.time()-start_time)))
    

In [ ]:
fp16_graph = tf.Graph()
with fp16_graph.as_default():
    inc=tf.constant(dummy_input, dtype=tf.float32)
    dataset=tf.data.Dataset.from_tensors(inc)
    dataset=dataset.repeat()
    iterator=dataset.make_one_shot_iterator()
    next_element=iterator.get_next()
    out = tf.import_graph_def(
      graph_def=trt_fp16_Graph,
      input_map={"import/input":next_element},
      return_elements=[ "import/resnet_v1_50/predictions/Softmax"]
    )
    out = out[0].outputs[0]

with tf.Session(graph=fp16_graph) as sess:
    print('Warming up for 10 batches...')
    for _ in range(10):
        sess.run(out)
    
    print('Benchmarking...')
    start_time = time.time()
    for _ in range(NUM_ITER):
        sess.run(out)
    print('Naitive FP32 Inference speed: %.2f samples/s'%(NUM_ITER*BATCH_SIZE/(time.time()-start_time)))
    

In [ ]:
int8_graph = tf.Graph()
with int8_graph.as_default():
    inc=tf.constant(dummy_input, dtype=tf.float32)
    dataset=tf.data.Dataset.from_tensors(inc)
    dataset=dataset.repeat()
    iterator=dataset.make_one_shot_iterator()
    next_element=iterator.get_next()
    out = tf.import_graph_def(
      graph_def=trt_int8_calibrated_graph,
      input_map={"import/input":next_element},
      return_elements=[ "import/resnet_v1_50/predictions/Softmax"],
      name=''        
    )
        
with tf.Session(graph=int8_graph) as sess:

    print('Warming up for 10 batches...')
    for _ in range(10):
        sess.run(out)
    
    print('Benchmarking...')
    start_time = time.time()
    for _ in range(NUM_ITER):
        sess.run(out)
    print('Naitive FP32 Inference speed: %.2f samples/s'%(NUM_ITER*BATCH_SIZE/(time.time()-start_time)))